# Introduction

# Preprocess

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
project = "line"
!export project=$project:line

In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2578, done.
remote: Counting objects: 100% (2578/2578), done.
remote: Compressing objects: 100% (2129/2129), done.
remote: Total 2578 (delta 644), reused 1432 (delta 417), pack-reused 0
Receiving objects: 100% (2578/2578), 32.50 MiB | 32.50 MiB/s, done.
Resolving deltas: 100% (644/644), done.


In [4]:
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1643309 sha256=026f11f71034815b7d22603026e8867897e81293ff66a4c96c289f2fb128da91
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezdmpf2z/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=ebd444d1ff9c8562d250cd1f96873bb449c017016b494d73a5abd2a9a4fc388b
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=de4125aa2c93117dcc1f374ddca635441f7d92dd8b27b44bde31e0471e878655
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=439a44def485a66e6635841278ecf156e7d34666327f257a633a

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [4]:
%cd /content/models/
!git clone https://github.com/L-Thirat/img_preprocess.git #Import Totoro and Nekobus data from Github

/content/models
Cloning into 'img_preprocess'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 50 (delta 18), reused 42 (delta 10), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [10]:
%cd /content/models/img_preprocess/
!python preprocess.py

/content/models/img_preprocess
{'obj': 'line', 'augment_num': 1000, 'val_ratio': 0.2, 'grayscale': True, 'img_resize': 412, 'fill_square': True, 'fill_square_color': 0, 'p_rotate': 0, 'rotate_angle_vari': 0, 'p_light': 0.3, 'light_vari': 80, 'p_noise': 0, 'noise_vari': 0, 'p_horizonal_flip': 0, 'p_vertical_flip': 0}
Augmenting frame_2021-02-12_201944_IN4.jpg ...
Augmenting frame_2021-02-12_201001_IN4.jpg ...
Augmenting frame_2021-02-12_201756_IN4.jpg ...
Augmenting frame_2021-02-12_211454_IN4.jpg ...
Augmenting frame_2021-02-12_205009_IN4.jpg ...
Augmenting frame_2021-02-12_204642_IN4.jpg ...
Augmenting frame_2021-02-12_202035_IN4.jpg ...
Augmenting frame_2021-02-12_211652_IN4.jpg ...
Augmenting frame_2021-02-12_210018_IN4.jpg ...
Augmenting frame_2021-02-12_205151_IN4.jpg ...
Augmenting frame_2021-02-12_211601_IN4.jpg ...
Augmenting frame_2021-02-12_213746_IN4.jpg ...
Augmenting frame_2021-02-12_214505_IN4.jpg ...
Augmenting frame_2021-02-12_220613_IN4.jpg ...
Augmenting frame_2021-02

In [35]:
%ls
%cd /content/
%ls

drive/  generate_tfrecord.py
/content
drive/  models/  sample_data/


In [48]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(float(member[4][0].text)),
                     int(float(member[4][1].text)),
                     int(float(member[4][2].text)),
                     int(float(member[4][3].text))
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
  data_dir = 'drive/MyDrive/projects/%s/data'%project
  if not os.path.exists(data_dir):
    os.makedirs(data_dir)

  tfrec_dir = 'drive/MyDrive/projects/%s/tfrecord'%project
  if not os.path.exists(tfrec_dir):
    os.makedirs(tfrec_dir)
  image_path = os.path.join(os.getcwd(), 'drive/MyDrive/projects/%s/preprocess/train'%project)
  xml_df = xml_to_csv(image_path)
  xml_df.to_csv('drive/MyDrive/projects/%s/data/train_labels.csv'%project, index=None)
  
  image_path = os.path.join(os.getcwd(), 'drive/MyDrive/projects/%s/preprocess/val'%project)
  xml_df = xml_to_csv(image_path)
  xml_df.to_csv('drive/MyDrive/projects/%s/data/val_labels.csv'%project,index=None)

main()

In [5]:
%mv /content/models/img_preprocess/data/object-detection.pbtxt /content/drive/MyDrive/projects/$project/data/object-detection.pbtxt

In [50]:
%cd /content/models/img_preprocess/tfrecord
!python generate_tfrecord.py --csv_input=../../../drive/MyDrive/projects/$project/data/train_labels.csv  --output_path=../../../drive/MyDrive/projects/$project/tfrecord/train.record --image_dir=../../../drive/MyDrive/projects/$project/preprocess/train
!python generate_tfrecord.py --csv_input=../../../drive/MyDrive/projects/$project/data/val_labels.csv  --output_path=../../../drive/MyDrive/projects/$project/tfrecord/val.record --image_dir=../../../drive/MyDrive/projects/$project/preprocess/val

/content/models/img_preprocess/tfrecord
2021-04-02 06:04:16.026867: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../../../drive/MyDrive/projects/line/preprocess/train
../.